In [179]:
from IPython.core.magic import register_cell_magic, cell_magic

@register_cell_magic
def parallel(line, cell):
    from os import environ
    parallel = bool(environ.get('PARALLEL', False))
    if parallel:
        get_ipython().run_cell_magic('px', '--local', cell)
    else:
        if '--only' in line:
            print('Skipping cell when not in parallel')
            return
        else:
            get_ipython().run_cell(cell)
            
@register_cell_magic
def comment(line, cell):
    '''Comment a cell.'''
    print('Skipping cell.')
    pass

In [169]:
%%parallel
print('Foo')

Foo


In [170]:
%%parallel --only
from IPython import parallel
c = parallel.Client()
view = c.load_balanced_view()

Skipping cell when not in parallel


In [171]:
%%parallel
%matplotlib inline
%load_ext Cython
%load_ext autoreload

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [216]:
%%parallel
%%cython -a
cimport cython
import numpy as np
cimport numpy as np
#from cython.parallel import prange

ctypedef double DTYPE_t

@cython.boundscheck(False)

def compute_inertia_tensor(np.ndarray[DTYPE_t, ndim=1] mass, np.ndarray[DTYPE_t, ndim=2] pos):
    '''Takes the mass and positions of particles and return the inertia tensor'''
    cdef int i, j
    cdef np.ndarray[DTYPE_t, ndim=2] I_t
    cdef np.ndarray[DTYPE_t, ndim=1] means
    cdef DTYPE_t tmp, mtot
    
    I_t = np.zeros((3, 3), dtype=np.double)
    mtot = np.sum(mass)
    
    # remove mean from positions
    means = np.mean(pos, 0)
    
    for j in range(pos.shape[0]):
        for i in range(3):
            pos[j, i] = pos[j, i] - means[i]
    
    for i in range(3):
        for j in range(i, 3):
            tmp = 0
            for k in range(pos.shape[0]):
                tmp = tmp + pos[k, i]*pos[k, j]
            I_t[i, j] = tmp
            I_t[j, i] = tmp
    return I_t

def project_points(x, y, z, a, b, c):
    """
    Projects the points with coordinates x, y, z onto the plane
    defined by a*x + b*y + c*z = 1
    
    From http://stackoverflow.com/questions/17836880/orthogonal-projection-with-numpy
    """
    vector_norm = a*a + b*b + c*c
    normal_vector = np.array([a, b, c]) / np.sqrt(vector_norm)
    point_in_plane = np.array([a, b, c]) / vector_norm

    points = np.column_stack((x, y, z))
    points_from_point_in_plane = points - point_in_plane
    proj_onto_normal_vector = np.dot(points_from_point_in_plane,
                                     normal_vector)
    proj_onto_plane = (points_from_point_in_plane -
                       proj_onto_normal_vector[:, None]*normal_vector)

    return point_in_plane + proj_onto_plane

cdef DTYPE_t correct(DTYPE_t el):
    '''Realign the particles when they're too far appart'''
    if el < 0.5:
        return el
    else:
        return el - 1.0
v_correct = np.vectorize(correct)

@cython.boundscheck(False)
def correct_particles(np.ndarray[DTYPE_t, ndim=2] particles):
    cdef int i, j
    cdef np.ndarray[DTYPE_t, ndim=1] maxis, minis
    maxis = np.max(particles, 0)
    minis = np.min(particles, 0)
    for i in range(3):
        if maxis[i] - minis[i] > 0.5:
            for j in range(particles.shape[1]):
                particles[j, i] = correct(particles[j, i])
    return particles

@cython.boundscheck(False)
cdef int binary_search(np.ndarray[int, ndim=1] a, int x, int left, int right):
    cdef int lo, hi, mid, midval
    lo = left
    hi = right
    while lo < hi:
        mid = (lo+hi)/2
        midval = a[mid]
        if midval < x:
            lo = mid+1
        elif midval > x: 
            hi = mid
        else:
            return mid
    return -1
        
def quicksearch_2(np.ndarray array, np.ndarray elements):
    '''Searches elements in array, where array and elements
    are both sorted'''
    cdef int left, right, pos
    positions = np.zeros(len(elements), dtype=int)
    left = 0
    right = len(array)
    for i in range(len(elements)):
        pos = binary_search(array, elements[i], left, right)
        if pos >-1:
            left = pos
        positions[i] = pos
    
    return positions

def quicksearch(np.ndarray[int, ndim=1] array, np.ndarray[int, ndim=1] elements):
    '''Returns the index of the elements in the array, asserting elements and array are ordered.'''
    cdef int i, ubound
    cdef np.ndarray[long, ndim=1] res, res1
    
    res = np.searchsorted(array, elements)
    
    ubound = len(array)

    for i in range(len(res)):
        if res[i] == 0 and elements[i] != array[res[i]]:
            res[i] = -1
        elif res[i] == ubound:
            res[i] = -1
    return res

def quicksearch_1(np.ndarray[int, ndim=1] array, np.ndarray[int, ndim=1] elements):
    cdef int i, ubound
    cdef np.ndarray[long, ndim=1] res0, res1
    res0 = np.searchsorted(array, elements, side='left')
    res1 = np.searchsorted(array, elements, side='right')
    return np.where(res1-res0 == 1, res0, -1)

In [217]:
masses = np.array([1, 1, 1], dtype=np.float)
pos = np.array([[1, 1, 1], [1, 2, 3], [1, 0, 2]], dtype=np.float)
%timeit compute_inertia_tensor_ref(masses, pos)
%timeit compute_inertia_tensor(masses, pos)
ref = compute_inertia_tensor_ref(masses, pos)
val = compute_inertia_tensor(masses, pos)
val

10000 loops, best of 3: 58.9 µs per loop
10000 loops, best of 3: 45.9 µs per loop


(False, array([[ 0.,  0.,  0.],
        [ 0.,  2.,  1.],
        [ 0.,  1.,  1.]]), array([[ 0.,  0.,  0.],
        [ 0.,  2.,  1.],
        [ 0.,  1.,  2.]]))

In [175]:
%%parallel
%autoreload
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tools as t
import pymses
import itertools
from tqdm import tqdm

import scipy.linalg as linalg
from scipy.interpolate import griddata
from scipy.stats import gaussian_kde

from sklearn.neighbors import KernelDensity

from cython_module import *


plt.rcParams['figure.figsize'] = (16, 9)
plt.rcParams['figure.dpi'] = 320

In [180]:
%%comment
%%px --local
def compute_inertia_tensor(mass,
                           pos):
    \'''Takes the mass and positions of particles and return the inertia tensor\'''
    #cdef int i, j
    #cdef np.ndarray[DTYPE_t, ndim=2] tmp, I_t
    I_t = np.zeros((3, 3))
    for i in range(3):
        for j in range(i, 3):
            tmp = np.sum(pos[:, i]*pos[:, j]) / np.sum(mass)
            I_t[i, j] = tmp
            I_t[j, i] = tmp
    return I_t

Skipping cell.


# Get inertia data

In [42]:
#%%px --local
halo_inertia = pd.read_csv('lists/halo.00002.inertia_tensor.dat', delim_whitespace=True).set_index('halo_id')

In [43]:
halo_inertia.head()

,xx,xy,xz,yx,yy,yz,zx,zy,zz
halo_id,,,,,,,,,
30246,0.048441,0.020570,0.089071,0.020570,0.009124,0.038115,0.089071,0.038115,0.165196
160957,0.478815,0.325352,0.588210,0.325352,0.221403,0.399873,0.588210,0.399873,0.723057
100666,0.097258,0.154665,0.023217,0.154665,0.246543,0.037000,0.023217,0.037000,0.005735
7,0.000475,0.001021,0.000344,0.001021,0.003022,0.000975,0.000344,0.000975,0.001175
191026,0.029886,0.011789,0.047257,0.011789,0.004684,0.018657,0.047257,0.018657,0.074841


# List of halos

In [44]:
#%%px --local
halo_list = pd.read_csv('lists/list_halo.dat',
                        delim_whitespace=True,
                        skiprows=1,
                        names=['id', 'level', 'mass', 'x', 'y', 'z', 'r']).set_index('id')

In [192]:
halo_list[halo_list.mass > 1e12]

,level,mass,x,y,z,r
id,,,,,,
7,1,1.329000e+12,0.031419,0.110498,0.993380,0.002035
40,1,1.253000e+12,0.004976,0.104755,0.018793,0.001995
66,1,3.020000e+12,0.991795,0.092866,0.055681,0.002674
68,1,1.828000e+12,0.033043,0.023285,0.041762,0.002263
102,1,3.295000e+13,0.944696,0.103903,0.081917,0.005927
206,1,2.495000e+12,0.020917,0.170988,0.002013,0.002508
207,1,1.442000e+12,0.028023,0.166578,0.023542,0.002089
257,1,4.041000e+13,0.965178,0.132439,0.968192,0.006343
269,1,8.662000e+12,0.084565,0.144323,0.962542,0.003800


# Galaxy to halo

In [46]:
#%%px --local
association = pd.read_csv('lists/associated_halogal_782.dat', delim_whitespace=True, skiprows=1,
                          names=['halo_id', 'level', 'halo_mass', 'gal_id', 'gal_mass']).set_index('halo_id')

In [47]:
association.head()

,level,halo_mass,gal_id,gal_mass
halo_id,,,,
2,1,7.121000e+11,14667,6.367000e+10
7,1,1.329000e+12,7271,1.012000e+11
10,1,6.511000e+11,15181,4.875000e+10
17,1,5.981000e+10,93879,4.056000e+08
18,1,2.100000e+11,72269,4.693000e+09


# Halo to cpu

In [48]:
#%%px --local
filename = 'lists/halo_to_cpu.00002.m<1e12.dat'
with open(filename, 'r') as f:
    rows, cols = [int(e) for e in f.readline().replace('\n', '').split()]
    names = ['halo_id'] + ['cpu_%i' % cpu for cpu in range(1, cols+1)] 

halo_to_cpu = pd.read_csv(filename, delim_whitespace=True, engine='c', skiprows=1,
                          names=names).set_index('halo_id')

In [49]:
halo_to_cpu[halo_to_cpu.cpu_1 > 0].head(20)

,cpu_1,cpu_2,cpu_3,cpu_4,cpu_5,cpu_6,cpu_7,cpu_8,cpu_9,cpu_10
halo_id,,,,,,,,,,
7,1,4,0,0,0,0,0,0,0,0
40,1,2,3,4,4093,0,0,0,0,0
66,1,2,3,4095,4096,4094,0,0,0,0
68,1,2,1627,1628,0,0,0,0,0,0
83,8,713,0,0,0,0,0,0,0,0
102,2,3,4038,4035,4067,4070,4068,4069,4081,4082
206,4,59,60,61,62,64,0,0,0,0
207,3,59,61,62,0,0,0,0,0,0
257,4,61,715,716,758,3378,3379,3381,3382,3383


# Get brick

In [50]:
#%%px --local
halos = t.io.read_brick('/data52/Horizon-AGN/TREE_DM_celldx2kpc_SC0.9r/tree_bricks782', low_mem=True)#, preload=True)

# Test with 1 halo of mass ~1e12

### Correct positions where required

### Project on 3 axis

## Estimate the inertia tensor

### Project on eigenvectors

## Estimate density using KDE
Take a look at https://jakevdp.github.io/blog/2013/12/01/kernel-density-estimation/ for details

# Compute the Inertia tensor

# Inverting from halo to cpu to cpu to halo

In [ ]:
cpu_to_halo = {}
for i in range(1, 4096+1):
    cpu_to_halo[i] = set()
for halo_i, cpus in tqdm(halo_to_cpu.iterrows()):
    for _, cpu in cpus.iteritems():
        if (cpu > 0):
            cpu_to_halo[cpu].add(halo_i)

# Computing inertia tensor for all halos (this may take a long time…)

In [94]:
#%%px --local
ro = pymses.RamsesOutput('/data52/Horizon-AGN/OUTPUT_DIR/', 2)
ro.verbose = False
parts = ro.particle_source(["pos", "mass", "id"])

Computing hilbert minimal domain description for output 2 ...
Done !


In [231]:
#%%px --local
step = 2
# Compute our meshgrid
xgrid = np.linspace(0, 1, step)
ygrid = np.linspace(0, 1, step)
zgrid = np.linspace(0, 1, step)

X, Y, Z = np.meshgrid(xgrid, ygrid, zgrid)

# Create a list of permutations 
comb_list = list(itertools.product(range(step), range(step), range(step)))

def get_halo_in_cpus(cpus):
    '''Return the halos that are contained in the cpus.'''
    halo_set = set()
    for cpu in cpus:
        halo_set.update(cpu_to_halo[cpu])
    return halo_set



In [245]:
halo_read = set()
def compute((i, j, k)):
    halo_inertia = pd.DataFrame(columns=['id', 'xx', 'xy', 'xz', 'yy', 'yz', 'zz',
                                         'complete', 'meanx', 'meany', 'meanz', 'nparts']).set_index('id')

    center = np.array([X[i,j,k], Y[i,j,k], Z[i,j,k]])

    region = pymses.filters.RegionFilter(pymses.utils.regions.Cube(center, 1.1/step), parts)
    dset   = region.flatten()

    #for halo_i, halo in tqdm(_tmp_h_list.iterrows()):
    for halo_i in get_halo_in_cpus(region._data_list):
        try:
            members = halos[halo_i-1]['members']
        except:
            print('Exception')
            continue
                        
        if halo_i in halo_read:
            #print('Skipping already read halo %s' % halo_i)
            continue

        ids = quicksearch(dset['id'], members)
        if len(ids) == 0:
            print('Empty !')
            continue
        pts = dset.points[ids]
        xyz = correct_particles(pts)
        masses = np.array(dset['mass'][ids])
        
        halo_inertia.at[halo_i, 'meanx'] = np.mean(xyz[:, 0])
        halo_inertia.at[halo_i, 'meany'] = np.mean(xyz[:, 1])
        halo_inertia.at[halo_i, 'meanz'] = np.mean(xyz[:, 2])
        halo_inertia.at[halo_i, 'nparts'] = len(ids)
        
        if len(ids) == len(halos[halo_i-1]['members']):
            halo_inertia.at[halo_i, 'complete'] = True
            halo_read.add(halo_i)
        else:
            halo_inertia.at[halo_i, 'complete'] = False
            print('Halo {} is incomplete, missing {:.2}%'.format(
                    halo_i, (100.*(len(halos[halo_i-1]['members']) - len(ids)) / (len(ids)))))
            continue


        I_t = compute_inertia_tensor(masses, xyz)
        l = ['x', 'y', 'z']
        for i in range(3):
            for j in range(i, 3):
                halo_inertia.at[halo_i, l[i]+l[j]] = I_t[i,j]

    return halo_inertia

p_results_p = view.map_sync(compute, comb_list[:5])

In [ ]:
p_results_p = [compute(ijk) for ijk in tqdm(comb_list)]

  0%|          | 0/8 [00:00<?, ?it/s]

Read and filter time : 7.46 s


 12%|█▎        | 1/8 [00:15<01:50, 15.81s/it]

Read and filter time : 17.68 s


 25%|██▌       | 2/8 [00:45<01:59, 19.89s/it]

Read and filter time : 13.45 s


 38%|███▊      | 3/8 [01:06<01:41, 20.22s/it]

Read and filter time : 14.91 s


 50%|█████     | 4/8 [01:27<01:21, 20.47s/it]

In [243]:
res = pd.DataFrame()
for p in p_results_p:
    res = res.append(p)

In [236]:
import pickle as pickle
with open('halo_inertia_m_mean.dump', 'w') as f:
    pickle.dump(p_results_p, f)

In [182]:
%%comment
import pickle as pickle
with open('halo_inertia_m_mean.dump', 'r') as f:
    p_results_p = pickle.load(f)

Skipping cell.


In [237]:
with open('inertia_m_mean.pickle', 'w') as f:
    pickle.dump(res, f)

In [244]:
res.sort_index()

,xx,xy,xz,yy,yz,zz,complete,meanx,meany,meanz,nparts
id,,,,,,,,,,,
7,4.00749,-0.518078,0.302562,9.58013,2.04956,1.93754,True,2.11351e-16,3.15361e-15,-3.63837e-15,16860
40,35.6973,-3.62566,19.4548,2.98453,-1.51902,13.4525,True,3.85628e-15,9.78663e-15,-5.82474e-15,15507
66,91.7637,-37.8092,94.6394,32.5887,-42.3273,99.0787,True,-6.21288e-15,-2.12778e-14,7.05092e-15,40019
68,81.0896,-8.85966,88.7269,2.92268,-9.1928,97.5078,True,5.78213e-14,1.24427e-15,2.31212e-14,23309
83,17.1636,-2.99549,27.9804,1.25203,-2.98517,63.4835,True,-3.50414e-15,4.90222e-15,8.15176e-16,12726
102,19.7537,-9.27527,20.3098,4.76854,-9.68876,21.0112,True,8.11479e-13,1.39894e-15,7.4417e-13,448056
206,24.7993,8.97122,13.6722,4.53554,4.76527,8.28065,True,1.96554e-13,8.64597e-14,-3.7136e-14,35819
207,6.90856,-16.505,-12.8569,49.5396,37.3132,30.2218,True,1.14813e-14,4.28789e-14,-2.91831e-14,19615
257,576.408,-328.098,658.924,202.409,-402.712,825.932,True,8.43765e-13,-1.08258e-13,-8.7046e-13,583376
